Scrapping du site France Travail pour les offres de data 

on récupère d'abord le nombre d'offres pour les jobs dans la data 

In [164]:
import re
import time
import random
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import sqlite3

def get_nombre_offres():
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    url='https://candidat.francetravail.fr/offres/recherche?motsCles=data&offresPartenaires=true&rayon=10&tri=0'

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser') 
        h1_element = soup.find('h1', class_='title')
        # Extraire le texte de la balise <h1>
        h1_text = h1_element.get_text()
        # Utiliser une expression régulière pour extraire le nombre
        nombre_offres = re.search(r'\d+', h1_text).group()   
        # Retourner le nombre d'offres
        print (nombre_offres)
        return int(nombre_offres)
    else:
        print("Erreur lors de la récupération de la page : HTTP", response.status_code)
        return None



La fonction buildurl va permettre de modifier l'url en fonction du nombre d'offres 


In [165]:
def build_url(limite_basse, limite_haute,recherche='data',partenaires="true",):
  
  
    base_url = "https://candidat.francetravail.fr/offres/recherche"
    params = {
      "motsCles": recherche,
      "offresPartenaires": partenaires,
      "range": f"{limite_basse}-{limite_haute}",
      "rayon": 10,
      "tri": 0
    }
    url = base_url + "?" + "&".join(f"{key}={value}" for key, value in params.items())
    return url



la fonction fetch job offer initie la connexion

In [166]:
def fetch_job_offers(url, headers):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Error fetching job offers: HTTP {response.status_code}")
        return None



parse job details va extraire les blocs de code html intéressants
extract job va extraire les infos des offres d'emploi


In [167]:
def parse_job_details(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    job_sections = soup.find_all('h2', class_='t4 media-heading')
    job_data = []
    for job_section in job_sections:
        job_details = extract_job_information(job_section)
        if job_details:
            job_data.append(job_details)
    return job_data

def extract_job_information(job_section):
  
    title = job_section.find('span', class_='media-heading-title').get_text(strip=True)
    subtext = job_section.find_next_sibling('p', class_='subtext')
    company = subtext.get_text(strip=True) if subtext else 'Information non disponible'
    location = subtext.find('span').get_text(strip=True) if subtext and subtext.find('span') else 'Lieu non disponible'
    description = job_section.find_next_sibling('p', class_='description').get_text(strip=True)
    contract = job_section.find_next_sibling('p', class_='contrat').get_text(strip=True)
    date_posted = job_section.find_next_sibling('p', class_='date').get_text(strip=True)     
    return {
          'poste': title,
          'entreprise': company,
          'lieu': location,
          'description': description,
          'contrat': contract,
          'date_publication': date_posted
      }




getoffres permet de récuperer les offres sur une page 

In [168]:
def get_offres(limite_basse, limite_haute, job_list):
    url = build_url(limite_basse, limite_haute)
    headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    html_content = fetch_job_offers(url, headers)
    if html_content:
        job_data = parse_job_details(html_content)
        job_list.extend(job_data)
    return job_list# Use extend to add elements to an existing list    return job_list
#liste=[]
#jobs=get_offres(0,20,liste)
#df=pd.DataFrame(jobs)

la fonction scrapFT sert à scrapper toutes les offres, le time sleep est généré entre 1 et 5 secondes pour simuler un comportement humain . La fonction retourne un dataframe

In [169]:
def scrap_FT():
    job_list=[]
    nb_max=get_nombre_offres()
    nb_max=41
    for i in range(0,nb_max,20):
        delai_attente = random.randint(1, 5)
        #time.sleep(delai_attente)
        get_offres(i, min(i+20, nb_max), job_list)
        
    if nb_max % 20 != 0:
        get_offres(nb_max-nb_max%20,nb_max,job_list)

    return pd.DataFrame(job_list)
    
df=scrap_FT()
print(df)
    

3583
                                                poste  \
0                                  Data Analyst (h/f)   
1                         Consultant Offre/Data (H/F)   
2                  DBA DEVELOPPEUR SQL BIG DATA (H/F)   
3                                  Data manager (F/H)   
4                                Tech Lead Data (H/F)   
5                                Tech Lead Data (H/F)   
6                                Tech Lead Data (H/F)   
7                                Tech Lead Data (H/F)   
8                   Consultant Data BI Confirmé (H/F)   
9                   Consultant Data BI Confirmé (H/F)   
10                  Consultant Data BI Confirmé (H/F)   
11                  Consultant Data BI Confirmé (H/F)   
12         Consultant Microsoft BI / Azure Data (H/F)   
13                                       Data analyst   
14                                       Data analyst   
15            Chef de projet Data Analyse REIMS (H/F)   
16            Chef de proj

On nettoie ensuite nos données en utilisant des dates absolues et suppression des caractères indésirables

In [170]:
def nettoyer_entreprise(df):
    # Utilise une expression régulière pour supprimer les caractères précédents et le tiret avant le nombre
    clean_df = df.str.split('\n').str[0]
    return clean_df

def nettoyer_contrat(df):
    # Divise la colonne "Entreprise" en fonction du caractère "\n" et ne conserve que la première partie
    clean_df = df.str.replace('\n','')
    return clean_df

def convert_to_date(duree):
    if "aujourd'hui" in duree:
        return datetime.now().strftime("%Y-%m-%d")
    elif "hier" in duree:
        return (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
    elif "il y a plus de" in duree:
        return (datetime.now() - timedelta(days=31)).strftime("%Y-%m-%d")
    elif "il y a" in duree:
        days_ago = int(duree.split()[4])
        return (datetime.now() - timedelta(days=days_ago)).strftime("%Y-%m-%d")

def nettoyage_total(df):
    df['entreprise'] = nettoyer_entreprise(df['entreprise'])
    df['contrat'] = nettoyer_contrat(df['contrat'])
    df["date_publication"] = df["date_publication"].apply(convert_to_date)
    return df
print(nettoyage_total(df))
    


                                                poste  \
0                                  Data Analyst (h/f)   
1                         Consultant Offre/Data (H/F)   
2                  DBA DEVELOPPEUR SQL BIG DATA (H/F)   
3                                  Data manager (F/H)   
4                                Tech Lead Data (H/F)   
5                                Tech Lead Data (H/F)   
6                                Tech Lead Data (H/F)   
7                                Tech Lead Data (H/F)   
8                   Consultant Data BI Confirmé (H/F)   
9                   Consultant Data BI Confirmé (H/F)   
10                  Consultant Data BI Confirmé (H/F)   
11                  Consultant Data BI Confirmé (H/F)   
12         Consultant Microsoft BI / Azure Data (H/F)   
13                                       Data analyst   
14                                       Data analyst   
15            Chef de projet Data Analyse REIMS (H/F)   
16            Chef de projet Da

On peut ensuite exporte les données dans un csv

In [171]:
def export_csv(dataframe, index=True):
    date_str = datetime.today().strftime("%Y-%m-%d-%H_%M")
    nom_fichier=f"france_travail_{date_str}.csv"
    dataframe.to_csv(nom_fichier, index=index)
    return f"exported to {nom_fichier}"

# Exemple d'utilisation
dataframe_to_csv(df)


Conversion du df en bdd

In [172]:
def conversion_df_bdd(df,connection=None):
    if  connection == None:
        connection = sqlite3.connect('ma_base.db')
    cursor = connection.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='offres'")
    table_existe = cursor.fetchone() is not None
    if not table_existe:
        cursor.execute("""
      CREATE TABLE offres (
        poste TEXT NOT NULL,
        entreprise TEXT NOT NULL,
        lieu TEXT,
        description TEXT,
        contrat TEXT,
        date_publication TEXT
      )
      """)
    try:
        data_to_insert = df[['poste', 'entreprise', 'lieu', 'description', 'contrat', 'date_publication']]
        data_to_insert.to_sql('offres', connection, if_exists='append', index=False)
        
        connection.commit()
        connection.close()
    except Exception as e:
        print(f"Une erreur est survenue lors de l'insertion des données : {e}")
    
conversion_df_bdd(df)

In [173]:
connection = sqlite3.connect('ma_base.db')

# Exécuter une requête SQL pour sélectionner toutes les lignes de la table "offres"
df_from_sql = pd.read_sql_query("SELECT * FROM offres", connection)

# Afficher le DataFrame contenant les données de la table "offres"
print(df_from_sql)

# Fermer la connexion à la base de données
connection.close()

                                          poste  \
0    Consultant Microsoft BI / Azure Data (H/F)   
1                                  Data analyst   
2                                  Data analyst   
3       Chef de projet Data Analyse REIMS (H/F)   
4       Chef de projet Data Analyse NANCY (H/F)   
..                                          ...   
110                         Data analyste (H/F)   
111                         Data analyste (H/F)   
112                              Data scientist   
113                         Data analyste (H/F)   
114                              Data scientist   

                            entreprise                 lieu  \
0                           ACT MOTION   75 - Paris (Dept.)   
1                               ABALON        Ile-de-France   
2                               ABALON        Ile-de-France   
3    GROUPEMENT D'INTERET PUBLIC PULSY           51 - REIMS   
4    GROUPEMENT D'INTERET PUBLIC PULSY           54 - NANCY   
..       